## Import Statements

In [1]:
import pandas as pd
import numpy as np

## Define functions

In [2]:
np.random.seed(seed=123456)

def get_initial_center_indices(size=None, k=None):
    '''
    size: total rows in data ( = highest index + 1)
    k = number of clusters
    '''
    return np.random.choice(a=size, replace=False, size=k)

def calculate_distance(a=None, b=None):
    '''
    a, b : numpy arrays of same length
    '''
    return np.sqrt(np.sum((a - b) ** 2))

def get_distance_from_each_center(x=None, centers=None):
    '''
    x : numpy array for the datapoint
    centers : pandas dataframe where each row represents center
    
    This function calculates distance of point x from all centers.
    '''    
    return centers.apply(calculate_distance, axis='columns', b=x)

def get_nearest_clusters_index(x=None, centers=None):
    '''
    x : numpy array for the datapoint
    centers : pandas dataframe where each row represents center
    
    This function calculates distance of point x from all centers.
    '''
    return np.argmin(get_distance_from_each_center(x, centers).values)

## Prepare data

In [3]:
data = pd.read_csv("~/Downloads/xclara.csv", header=0)
data = data.sample(n=100, random_state=12345)
data.reset_index(inplace=True, drop=True)

## Inefficient KMeans!

In [4]:
def kmeans_my_1(data=None, k=3, max_iter=200):
    import copy
    
    centers = data.loc[np.random.choice(data.shape[0], replace=False, size=k)].reset_index(inplace=False, drop=True)
    cur_iter = 0
    shouldIterate = True

    while shouldIterate:
        clusters = data.apply(get_nearest_clusters_index, axis='columns', centers=centers)
        old_centers = copy.deepcopy(centers)
        
        # update centers
        for i in range(k):
            centers.loc[i] = data.loc[clusters == i].mean()

        cur_iter += 1
        shouldIterate = any(np.sqrt(np.sum((centers - old_centers) ** 2)) > 1) and cur_iter < max_iter

    print(centers)
    print(cur_iter)

## Our solution is accurate, but sklearn is FAST! 
~50xfaster!

In [5]:
from sklearn.cluster import KMeans
from datetime import datetime

km = KMeans(n_clusters=3)

start = datetime.now()
km.fit(data)
sklearn_time = datetime.now() - start
print("sklearn time : {}".format(sklearn_time))

print(km.cluster_centers_)

start = datetime.now()
kmeans_my_1(data=data, k=3)
our_time = datetime.now() - start
print("solution 1 time : {}".format(our_time))
print("sklearn is faster by {0:.0f}x".format(our_time / sklearn_time))


sklearn time : 0:00:00.042014
[[41.36099029 60.95182088]
 [67.50411    -8.87655433]
 [ 9.01868806  8.14900125]]
          V1         V2
0  67.504110  -8.876554
1  41.360990  60.951821
2   9.018688   8.149001
4
solution 1 time : 0:00:01.811799
sklearn is faster by 43x


### Let us try to reduce use of dataframe apply function.

In [6]:
def kmeans_my_2(data=None, k=3, max_iter=200):
    import copy
    
    centers = data.loc[np.random.choice(data.shape[0], replace=False, size=k)].values
    clusters = np.array([-1] * data.shape[0])
    cur_iter = 0
    shouldIterate = True

    while shouldIterate:
        for nth_row in range(data.shape[0]):
            data_row = data.loc[nth_row]
            
            nearest_cluster = -1
            min_distance = float("inf")
            
            for cluster_num in range(k):
                distance = np.sum((centers[cluster_num] - data_row) ** 2)
                if distance < min_distance:
                    nearest_cluster = cluster_num
                    min_distance = distance
            
            clusters[nth_row] = nearest_cluster
                    
        old_centers = copy.deepcopy(centers)
        
        # update centers
        for i in range(k):
            centers[i] = data.loc[clusters == i].mean().values

        cur_iter += 1
        
        if cur_iter >= max_iter:
            break
        
        shouldIterate = False
        
        for i in range(centers.shape[0]):
            if np.sum((centers[i] - old_centers[i]) ** 2)  > 1:
                shouldIterate = True
                break

    print(centers)
    print(cur_iter)

## Still sklearn is ~25x faster

In [7]:
from sklearn.cluster import KMeans
from datetime import datetime

km = KMeans(n_clusters=3)

start = datetime.now()
km.fit(data)
sklearn_time = datetime.now() - start
print("sklearn time : {}".format(sklearn_time))

print(km.cluster_centers_)

start = datetime.now()
kmeans_my_2(data=data, k=3)
our_time = datetime.now() - start
print("solution 2 time : {}".format(our_time))
print("sklearn is faster by {0:.0f}x".format(our_time / sklearn_time))


sklearn time : 0:00:00.040630
[[67.50411    -8.87655433]
 [41.36099029 60.95182088]
 [ 9.01868806  8.14900125]]
[[ 9.01868806  8.14900125]
 [67.50411    -8.87655433]
 [41.36099029 60.95182088]]
5
solution 2 time : 0:00:01.483758
sklearn is faster by 37x


### Since clusters calculation is bottle neck, let us use numpy there.

In [8]:
def kmeans_my_3(data=None, k=3, max_iter=200):
    import copy
    max_iter = 200
    k=3
    
    centers = data.loc[np.random.choice(data.shape[0], replace=False, size=k)].values
    cur_iter = 0
    shouldIterate = True
    clusters = np.array([-1] * data.shape[0])

    while shouldIterate:
        for nth_row in range(data.shape[0]):
            clusters[nth_row] = np.argmin(np.sum((centers - data.loc[nth_row].values) ** 2, axis=1))
                    
        old_centers = copy.deepcopy(centers)
        
        # update centers
        for i in range(k):
            centers[i] = data.loc[clusters == i].mean().values

        cur_iter += 1
        
        if cur_iter >= max_iter:
            break
        
        shouldIterate = any(np.sum((centers - old_centers) ** 2, axis=1) > 1)

    print(centers)
    print(cur_iter)

### With the use of numpy for norm, we are now ~5x slower than sklearn

In [9]:
from sklearn.cluster import KMeans
from datetime import datetime

km = KMeans(n_clusters=3)

start = datetime.now()
km.fit(data)
sklearn_time = datetime.now() - start
print("sklearn time : {}".format(sklearn_time))

print(km.cluster_centers_)

start = datetime.now()
kmeans_my_3(data=data, k=3)
our_time = datetime.now() - start
print("solution 3 time : {}".format(our_time))
print("sklearn is faster by {0:.0f}x".format(our_time / sklearn_time))


sklearn time : 0:00:00.042791
[[67.50411    -8.87655433]
 [ 9.01868806  8.14900125]
 [41.36099029 60.95182088]]
[[ 9.01868806  8.14900125]
 [41.36099029 60.95182088]
 [67.50411    -8.87655433]]
3
solution 3 time : 0:00:00.153458
sklearn is faster by 4x
